# Exploratory Data Analysis

In [ ]:
pwd

## Data Preparation and Dependencies Setup

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import os
from scipy.stats import stats

# Study data files
seasonal_pollution_path = os.path.join("..","Resources", "Data", "seasonal_combined_df1_ww.csv")
monthly_pollution_path = os.path.join("..","Resources", "Data", "monthly_combined_df1_ww.csv")

# Read the data and the results

seasonal_pollution_df = pd.read_csv(seasonal_pollution_path)
monthly_pollution_df = pd.read_csv(monthly_pollution_path)

# Create output directory
stats_output_dir_season = 'Stats_Outputs/Season'
if not os.path.exists(stats_output_dir_season):
    os.makedirs(stats_output_dir_season)

stats_output_dir_month = 'Stats_Outputs/Month'
if not os.path.exists(stats_output_dir_month):
    os.makedirs(stats_output_dir_month)


## Seasonal EDA

In [ ]:
seasonal_pollution_df.head()


In [ ]:
dtypes = seasonal_pollution_df.dtypes
dtypes

In [ ]:
seasonal_pollution_df.describe()

In [ ]:
# Check for missing values
print(seasonal_pollution_df.isnull().sum())

In [ ]:
# Checking the number of mice.
number_of_cities=seasonal_pollution_df['city'].nunique()
number_of_cities

In [ ]:
# Create a new column by combining 'city' and 'Season_Year' with an underscore
seasonal_pollution_df['city_season'] = seasonal_pollution_df['city'] + '_' + seasonal_pollution_df['Season_Year']

# Check the first few rows to ensure the new column is correct
print(seasonal_pollution_df[['city_season']].head())



In [ ]:
entry_count = len(seasonal_pollution_df['city_season'])
entry_count

In [ ]:
# Get the duplicate city
duplicates=seasonal_pollution_df[seasonal_pollution_df.duplicated(subset=["city", "Season_Year"], keep=False)]
duplicates['city'].unique()

In [ ]:
duplicated_rows=seasonal_pollution_df[seasonal_pollution_df.duplicated(subset=['city', 'Season_Year'], keep=False)]

print('Duplicated entries: ', len(duplicated_rows))
print(duplicated_rows.head())


In [ ]:
# Dropping duplicates, keeping the first entry
seasonal_pollution_clean_df = seasonal_pollution_df.drop_duplicates(subset=['city', 'Season_Year'], keep='first')
seasonal_pollution_clean_df.head()

In [ ]:
# Checking the number of mice.
number_of_cities_no_dups=seasonal_pollution_clean_df['city'].nunique()
number_of_cities_no_dups

In [ ]:
entry_count = len(seasonal_pollution_clean_df['city_season'])
entry_count

In [ ]:
# List of columns to plot
pollutants = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']

# Create a histogram for each column
for pollutant in pollutants:
    plt.figure()
    seasonal_pollution_df[pollutant].plot(kind='hist', bins=20)
    plt.xlabel(pollutant)
    plt.ylabel('Frequency')
    plt.title(f'Histogram of global {pollutant}')
    plt.savefig(f'{stats_output_dir_season}/{pollutant}_histogram_global.png')
    plt.close()


In [ ]:
seasons = seasonal_pollution_clean_df['Season'].unique()


In [ ]:
# Optional: Define a color map for different seasons for visual clarity
colors = ['blue', 'green', 'red', 'purple']  # One color for each season
season_color = {season: color for season, color in zip(seasons, colors)}

for pollutant in pollutants:
    plt.figure(figsize=(10, 6))  # Create a figure for each pollutant

    for season in seasons:
        season_data = seasonal_pollution_df[seasonal_pollution_df['Season'] == season][pollutant].dropna()
        plt.hist(season_data, bins=20, histtype='step', linewidth=1.5, label=season, color=season_color[season])

    plt.title(f'Global Histogram of {pollutant.capitalize()} by Season')
    plt.xlabel(f'{pollutant.capitalize()} Concentration')
    plt.ylabel('Frequency')
    plt.legend(title='Season')
    plt.grid(True)  # Optional: Turn on grid for better readability

    # Save the plot in the output directory
    plt.savefig(f'{stats_output_dir_season}/{pollutant}_global_histogram_by_season.png')
    plt.close()  # Close the plot to free up memory

In [ ]:
pollutants = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
for pollutant in pollutants:
    seasonal_pollution_clean_df[pollutant] = pd.to_numeric(seasonal_pollution_clean_df[pollutant], errors='coerce')

mean_pollution = seasonal_pollution_clean_df.groupby(['Season'])[pollutants].mean()
max_pollution = seasonal_pollution_clean_df.groupby(['Season'])[pollutants].max()
min_pollution = seasonal_pollution_clean_df.groupby(['Season'])[pollutants].min()
std_pollution = seasonal_pollution_clean_df.groupby(['Season'])[pollutants].std()

# Concatenate the statistics DataFrames along columns
stats_by_season = pd.concat([
    mean_pollution.add_suffix('_Mean'),
    max_pollution.add_suffix('_Max'),
    min_pollution.add_suffix('_Min'),
    std_pollution.add_suffix('_Std')
], axis=1)

# Transpose the DataFrame
transposed_stats_by_season = stats_by_season.T

# Sort the DataFrame by the index directly if 'Season' is the index
sorted_transposed_stats_by_season = transposed_stats_by_season.sort_index(ascending=True)

# Print the sorted DataFrame
print(sorted_transposed_stats_by_season)

In [ ]:
# IQR plots

# Output file path
txt_file_path = os.path.join(stats_output_dir_season, 'IQR.txt')

# Open the output file and write to it
with open(txt_file_path, 'w') as txt_file:
    # Loop over each season
    for season in seasonal_pollution_clean_df['Season'].unique():
        txt_file.write(f"Season: {season}\n")
        # Filter data for the current season
        season_data = seasonal_pollution_clean_df[seasonal_pollution_clean_df['Season'] == season]

        # Calculate the IQR for each pollutant for the current season
        for pollutant in pollutants:
            if pollutant in season_data.columns:  # Check if the column exists
                clean_data = season_data[pollutant].dropna()
                quartiles = clean_data.quantile([.25, .75])
                first_quartile = quartiles[0.25]
                third_quartile = quartiles[0.75]
                iqr = third_quartile - first_quartile

                # Calculate lower and upper bounds to identify outliers
                lower_bound = first_quartile - 1.5 * iqr
                upper_bound = third_quartile + 1.5 * iqr
                outliers = clean_data[(clean_data < lower_bound) | (clean_data > upper_bound)]

                txt_file.write(f"{pollutant.capitalize()} IQR: {iqr:.2f}\n")
                txt_file.write(f"Values below {lower_bound:.2f} or above {upper_bound:.2f} could be outliers.\n")

                # Identify and report potential outliers
                if not outliers.empty:
                    txt_file.write(f"Potential outliers: {outliers.values}\n")
                else:
                    txt_file.write("No potential outliers detected.\n")
                txt_file.write("================================================================================\n")

        txt_file.write("\n")  # Add a newline for separation between seasons
    



In [ ]:

# Assuming 'seasonal_pollution_df', 'seasons', and 'pollutants' are predefined
# Make sure the output directory exists
os.makedirs(stats_output_dir_season, exist_ok=True)

# Prepare data for plotting
pollutant_data_by_season = {
    pollutant: [
        seasonal_pollution_df[
            (seasonal_pollution_df['Season'] == season) & 
            (seasonal_pollution_df[pollutant].notna())
        ][pollutant] for season in seasons
    ] for pollutant in pollutants
}

# Create box plots for each pollutant
for pollutant, data in pollutant_data_by_season.items():
    fig, ax = plt.subplots()
    ax.boxplot(data, labels=seasons, showfliers=False)
    ax.set_title(f'{pollutant.capitalize()} Levels Across Seasons')
    ax.set_ylabel('Concentration')
    ax.set_xlabel('Season')

    # Setting x-axis tick labels with rotation and applying tight layout
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Define output file name for each pollutant plot and save it
    png_file_name = f'{pollutant}_box_plots.png'
    png_file_path = os.path.join(stats_output_dir_season, png_file_name)

    # Save the plot to the specified path
    fig.savefig(png_file_path)
    
    # Optionally display the plot
    plt.show()

In [ ]:
# ANOVA - nh3

group0 = seasonal_pollution_clean_df[seasonal_pollution_clean_df["Season"] == 'Summer']["nh3"]
group1 = seasonal_pollution_clean_df[seasonal_pollution_clean_df["Season"] == 'Autumn']["nh3"]
group2 = seasonal_pollution_clean_df[seasonal_pollution_clean_df["Season"] == 'Winter']["nh3"]
group3 = seasonal_pollution_clean_df[seasonal_pollution_clean_df["Season"] == 'Spring']["nh3"]

In [ ]:
# Perform the ANOVA - nh3
stats.f_oneway(group0, group1, group2, group3)

In [ ]:
# Make sure the output directory exists
os.makedirs(stats_output_dir_season, exist_ok=True)

# Prepare data for ANOVA and plotting
pollutant_data_by_season = {
    pollutant: [
        seasonal_pollution_df[
            (seasonal_pollution_df['Season'] == season) & 
            (seasonal_pollution_df[pollutant].notna())
        ][pollutant].values for season in seasons
    ] for pollutant in pollutants
}

# Prepare a file to log ANOVA results
anova_results_path = os.path.join(stats_output_dir_season, 'ANOVA_results.txt')
with open(anova_results_path, 'w') as results_file:
    results_file.write("ANOVA Results for Pollutants Across Seasons\n")
    results_file.write("-------------------------------------------------\n")

    # Conduct ANOVA for each pollutant and create box plots
    for pollutant, data in pollutant_data_by_season.items():
        # Ensure there's enough data for ANOVA (i.e., at least one season with data)
        if all(len(d) > 1 for d in data):  # Checking that each season has more than one data point
            # Perform ANOVA
            f_stat, p_value = stats.f_oneway(*data)
            results_file.write(f"{pollutant.capitalize()}:\n")
            results_file.write(f"F-Statistic: {f_stat:.4f}, P-Value: {p_value:.4e}\n")
            results_file.write("\n")

            # Create and save box plot
            fig, ax = plt.subplots()
            ax.boxplot(data, labels=seasons, showfliers=False)
            ax.set_title(f'{pollutant.capitalize()} Levels Across Seasons')
            ax.set_ylabel('Concentration')
            ax.set_xlabel('Season')
            plt.xticks(rotation=45)
            plt.tight_layout()
            png_file_name = f'{pollutant}_box_plots.png'
            png_file_path = os.path.join(stats_output_dir_season, png_file_name)
            fig.savefig(png_file_path)
            plt.close(fig)  # Close the figure to free up memory

        else:
            results_file.write(f"Not enough data for ANOVA on {pollutant}.\n\n")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from your ANOVA results
pollutants = ['CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2_5', 'PM10', 'NH3']
f_statistics = [30.5779, 21.9336, 44.9542, 198.0145, 7.5631, 25.4162, 7.9426, 17.9767]
p_values = [1.5974e-19, 4.4627e-14, 1.5603e-28, 3.4625e-119, 4.8375e-05, 2.8428e-16, 2.8098e-05, 1.3981e-11]

# Convert P-values to a negative log scale to make them more readable on the graph
neg_log_p_values = -np.log10(p_values)


In [ ]:
fig, ax = plt.subplots()

# Create a bar chart for the F-statistics
bars = ax.bar(pollutants, f_statistics, color='skyblue')

# Label the chart and axes
ax.set_xlabel('Pollutant')
ax.set_ylabel('F-Statistic')
ax.set_title('ANOVA F-Statistics for Pollutants Across Seasons')
ax.set_yscale('log')  # Use a log scale for better visibility if values vary widely

# Annotate bars with P-values
for bar, p_value in zip(bars, neg_log_p_values):
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval, f'{p_value:.2f}', va='bottom', ha='center')  # Adjust formatting as needed

plt.xticks(rotation=45)  # Rotate labels for better readability
plt.tight_layout()  # Adjust layout to fit everything
plt.show()


In [ ]:
# Data from your ANOVA results (reusing some of the example data)
pollutants = ['CO', 'NO', 'NO2', 'O3', 'SO2', 'PM2_5', 'PM10', 'NH3']
p_values = [1.5974e-19, 4.4627e-14, 1.5603e-28, 3.4625e-119, 4.8375e-05, 2.8428e-16, 2.8098e-05, 1.3981e-11]

# Convert P-values to a negative log scale to make them more readable on the graph
neg_log_p_values = -np.log10(p_values)

# Create the plot
fig, ax = plt.subplots()

# Create a bar chart for the negative log of P-values
bars = ax.bar(pollutants, neg_log_p_values, color='skyblue')

# Label the chart and axes
ax.set_xlabel('Pollutant')
ax.set_ylabel('-log10(P-Value)')
ax.set_title('Negative Log of P-Values for ANOVA Tests Across Seasons')

# Setting y-axis with a log scale may not be necessary here because values are already transformed
# ax.set_yscale('log')

# Annotate bars with original P-values
for bar, p_value in zip(bars, p_values):
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval, f'{p_value:.2e}', va='bottom', ha='center')  # Adjust formatting as needed

plt.xticks(rotation=45)  # Rotate labels for better readability
plt.tight_layout()  # Adjust layout to fit everything
plt.show()

Analysis Summary:

1. Carbon Monoxide (CO):
- F-Statistic: 30.5779
- P-Value: 1.5974e-19
- Interpretation: The extremely small P-value suggests there is a highly significant difference in the mean levels of CO across seasons.

2. Nitric Oxide (NO):
- F-Statistic: 21.9336
- P-Value: 4.4627e-14
- Interpretation: There is a significant difference in NO concentrations across seasons, indicated by a very small P-value.

3. Nitrogen Dioxide (NO2):
- F-Statistic: 44.9542
- P-Value: 1.5603e-28
- Interpretation: The results show a highly significant seasonal variation in NO2 levels.

4. Ozone (O3):
- F-Statistic: 198.0145
- P-Value: 3.4625e-119
- Interpretation: Ozone shows the strongest seasonal variation among all pollutants tested, with a strikingly significant difference in means.

5. Sulfur Dioxide (SO2):
- F-Statistic: 7.5631
- P-Value: 4.8375e-05
- Interpretation: There is a significant difference in SO2 concentrations across different seasons, although the effect size (as suggested by the F-statistic) is smaller compared to others like O3 or NO2.

6. Particulate Matter 2.5 (PM2.5):
- F-Statistic: 25.4162
- P-Value: 2.8428e-16
- Interpretation: PM2.5 levels vary significantly across seasons, with a very low P-value indicating strong statistical significance.

7. Particulate Matter 10 (PM10):
- F-Statistic: 7.9426
- P-Value: 2.8098e-05
- Interpretation: There's a significant seasonal variation in PM10 levels, similar to SO2 in terms of statistical weight.

8. Ammonia (NH3):
- F-Statistic: 17.9767
- P-Value: 1.3981e-11
- Interpretation: NH3 concentrations significantly differ across seasons, showing substantial variation.

```Conclusion:```
The ANOVA tests reveal significant differences in the seasonal concentrations of all tested pollutants. The results suggest that factors influencing these pollutants vary across different times of the year, which could be due to changes in weather, heating practices, traffic patterns, or other seasonal activities affecting pollutant levels.

### Spatial Plotting

In [ ]:
# Import the required libraries
import hvplot.pandas
import pandas as pd

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [ ]:
coordinates_df = seasonal_pollution_clean_df.copy()

# Columns of interest for spatial plotting
coordinates_df = coordinates_df[['city', 'country_full', 'latitude', 'longitude', 'Season', 'population', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']]

coordinates_df.head(3)

In [ ]:
# Select only numeric columns for aggregation
numeric_cols = coordinates_df.select_dtypes(include=[np.number]).columns.tolist()

# Specify the columns to group by
grouping_cols = ['city', 'Season']

# Group by 'city' and 'Season', and calculate the mean for numeric columns
city_season_aggregates = coordinates_df.groupby(grouping_cols)[numeric_cols].mean()

# Reset the index if you want 'city' and 'Season' as columns and not as an index
city_season_aggregates = city_season_aggregates.reset_index()

city_season_aggregates.head()  # Shows the first few rows of the aggregated data

In [ ]:
import geoviews as gv
import geoviews.tile_sources as gts
import hvplot.pandas  # Provides an extension to convert Pandas DataFrame into a Holoviews object
import pandas as pd
import holoviews as hv

In [ ]:
# Enable Holoviews extension for Jupyter
hv.extension('bokeh')

# Select a tile source
tiles = gv.tile_sources.OSM

In [ ]:
# CO Pollutant Levels by Season (Winter) and City - testing one

# Filter the DataFrame for a specific season, e.g., 'Winter'
winter_data = coordinates_df[coordinates_df['Season'] == 'Winter']

# Define the color scale range
color_range = (0, 1000)  # Adjusted to limit maximum CO level to 1400

# Plotting code as previously discussed, with clim parameter added
points = winter_data.hvplot.points('longitude', 'latitude', geo=True, color='co', cmap='rainbow', size=(winter_data['population']*0.0001), hover_cols=['city', 'Season', 'co'], clim=color_range)
plot = tiles * points
plot.opts(width=700, height=500, toolbar='above', title="Winter Pollution Levels by City")

In [ ]:
# Define the list of pollutants and seasons
pollutants = ['co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3']
seasons = ['Winter', 'Spring', 'Autumn', 'Summer']

# Iterate over pollutants and seasons
for pollutant in pollutants:
    for season in seasons:
        # Filter the DataFrame for the current season and pollutant
        data = coordinates_df[(coordinates_df['Season'] == season)]
        
        # Define the color scale range for the current pollutant
        color_range = (0, 1000)  # Adjusted to limit maximum CO level to 1000

        # Plotting code with population as bubble size and current pollutant
        points = data.hvplot.points('longitude', 'latitude', geo=True, color=pollutant, cmap='rainbow', size=(data['population'] * 0.0001), hover_cols=['city', 'Season', pollutant], clim=color_range)
        plot = tiles * points
        plot.opts(width=700, height=500, toolbar='above', title=f"{season} Pollution Levels by City")

        # Save the plot as a PNG file in the outputs directory
        output_filename = os.path.join(stats_output_dir_season, f"{pollutant}_{season}.png")
        hv.save(plot, output_filename)

        print(f"Plot saved for {pollutant} in {season} season.")

### Additional Analysis

In [ ]:
import plotly.express as px

colour=['orange', 'green', 'yellow', 'blue']

# Assuming 'mean_pollution' is already computed and properly structured
fig = px.bar(mean_pollution.reset_index(), x='Season', y='co', color='Season', barmode='group', color_discrete_sequence=colour,
             title="Average CO Levels by Season and City")
fig.show()

In [ ]:
print(mean_pollution.head())
print(mean_pollution.dtypes)

In [ ]:
# Select the top 10 cities based on CO levels, replace 'co' with the correct column name if different
top_cities = mean_pollution.nlargest(10, 'co').index

# Print the top cities to verify
print("Top cities based on CO levels:", top_cities)

In [ ]:
# Filter your main DataFrame to include only the top cities
filtered_data = seasonal_pollution_df[season['city'].isin(top_cities)]

# Assuming your DataFrame has a 'Season' column, and you want to visualize CO levels
import matplotlib.pyplot as plt

# Calculate seasonal means for the filtered top cities
seasonal_means = filtered_data.groupby(['Season', 'city'])['co'].mean().unstack()

# Plotting
seasonal_means.plot(kind='bar', figsize=(14, 7))
plt.title('Average CO Levels by Season for Top 10 Cities')
plt.xlabel('Season')
plt.ylabel('Average CO Level')
plt.legend(title='City', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()  # Adjust layout to make room for the legend
plt.show()

